In [1]:
#import sys
#from osgeo import gdal
import fiona
import pandas as pd

PROJ: proj_create_from_database: Cannot find proj.db


In [2]:
data_path = '../../../../Praktikum_data/geodata_vienna/'

## Export sectors upper left anf lower right coordinates

In [3]:
sectors = fiona.open(data_path + 'MZKBLATT5000OGD/MZKBLATT5000OGDPolygon.shp')
#print(next(iter(sectors)))
data_list = []
data = []
for sector in sectors:

    data.append(sector['id'])
    data.append(sector['properties']['BNR5000'])
    
    # sectors are defined with 21 points (as elipsoids, 1st and 21st is the same)
    # if 1st point (idx 0) is one of the corners (it seems to be like that), all the corners are saved with 0(=20), 5, 10 and 15 idx
    #corners = sector['geometry']['coordinates'][0][0], sector['geometry']['coordinates'][0][5], sector['geometry']['coordinates'][0][10], sector['geometry']['coordinates'][0][15]
    corners = sector['geometry']['coordinates'][0][:-1:5]
    
    left = sorted(corners)[:2]
    upper_left = left[0]
    if left[0][1] < left[1][1]:
        upper_left = left[1]
        
    right = sorted(corners)[2:]
    lower_right = right[0]
    if right[0][1] > right[1][1]:
        lower_right = right[1]
        
    #print(upper_left[0], upper_left[1], lower_right[0], lower_right[1])
    
    data.append(upper_left[0])#min_x
    data.append(upper_left[1])#max_y
    data.append(lower_right[0])#max_x
    data.append(lower_right[1])#min_y

    data_list.append(data)
    data = []
#print(data_list)

In [4]:
df = pd.DataFrame(data_list, columns = ['id', 'sector', 'upper_left_x', 'upper_left_y', 'lower_right_x', 'lower_right_y'])
df.to_json(data_path + r'vienna_sectors.json', orient='index')

In [5]:
sectors = pd.read_json(data_path + r'vienna_sectors.json', orient='index')
sectors

,id,sector,upper_left_x,upper_left_y,lower_right_x,lower_right_y
0,0,12/2,16.163534,48.333177,16.197313,48.310740
1,1,13/1,16.197252,48.333222,16.231016,48.310775
2,2,13/2,16.230970,48.333258,16.264719,48.310801
3,3,14/1,16.264688,48.333284,16.298423,48.310817
4,4,14/2,16.298406,48.333300,16.332126,48.310823
...,...,...,...,...,...,...
145,145,57/4,16.500068,48.130843,16.533567,48.108307
146,146,58/3,16.533653,48.130790,16.567138,48.108244
147,147,58/4,16.567239,48.130727,16.600708,48.108172
148,148,59/3,16.600824,48.130655,16.634279,48.108089
